# LLM Optimization Modelling Experiment

In [4]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [5]:
problem = '''A buyer needs to acquire 239,600,480 units of a product and is considering bids from five suppliers, labeled A through E, each of whom can only supply a portion of the total required amount.
Each vendor has proposed different pricing structures, incorporating both setup fees and variable unit costs that change based on the quantity ordered.

The buyer's objective is to allocate the order among these suppliers to minimize overall costs, accounting for both setup and unit costs.

Vendor A offers a set up cost of $3855.34 and a unit cost of $61.150 per thousand of units.
Vendor A can supply up to 33 million units.

Vendor B offers a set up cost of $125,804.84 if purchasing between 22,000,000-70,000,000 units from vendor B with a unit cost of $68.099 per thousand units.
If purchasing between 70,000,001-100,000,000 units from vendor B, the set up cost increases to $269304.84 and the unit cost sinks to $66.049 per thousand units.
If purchasing between 100,000,001-150,000,000 units from vendor B, the unit cost per thousand units further decreases to $64.099, but the set up cost increases to $464304.84.
If purchasing between 150,000,001 and 160,000,000 units from vendor B, the unit cost is $62.119 per thousand units and the set up cost equals $761304.84.

Vendor C offers set up costs of $13,456.00 and a unit cost of $62.019 per thousand units.
Vendor C can supply up to 165.6 million units. Vendor D offers set up costs of $6,583.98 and a unit cost of $72.488 for a set of thousand units.

Vendor D can supply up to 12 million units at a price of $72.488 per thousand units and with a set up cost of $6583.98.

Vendor E offers free set up if purchasing between 0 and 42 million units of vendor E with a unit price of $70.150 per thousand units.
If purchasing between 42,000,001 and 77 million units from vendor E, the unit cost starts at $68.150 per thousand units, but with every one million units purchased the price decreases at a rate of 0.05 percent. An additional set up cost of $84000 will be charged as well.

Note that zero units may be purchased from vendor B: otherwise no positive number of units less than 22,000,000 may be purchased. 
'''

## 2. Generate the mathematical model

In [13]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [14]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Supplier Selection

**Parameters:**

* **D:** Total demand (units): 239,600,480 
* **S_i:** Setup cost for vendor i (i = A, B, C, D, E)
   * S_A = $3855.34
   * S_B1 = $125,804.84 (22M - 70M units)
   * S_B2 = $269,304.84 (70M+1 - 100M units)
   * S_B3 = $464,304.84 (100M+1 - 150M units)
   * S_B4 = $761,304.84 (150M+1 - 160M units)
   * S_C = $13,456.00
   * S_D = $6,583.98
   * S_E1 = $0 (0 - 42M units)
   * S_E2 = $84,000 (42M+1 - 77M units)
* **C_ij:** Unit cost per thousand units for vendor i in price range j 
   * C_A = $61.150
   * C_B1 = $68.099 (22M - 70M units)
   * C_B2 = $66.049 (70M+1 - 100M units)
   * C_B3 = $64.099 (100M+1 - 150M units)
   * C_B4 = $62.119 (150M+1 - 160M units)
   * C_C = $62.019
   * C_D = $72.488
   * C_E1 = $70.150 (0 - 42M units)
   * C_E2_base = $68.150 (42M+1 - 77M units)
   * C_E2_discount = 0.0005 (per thousand units over 42M)
* **Cap_i:** Maximum supply capacity for vendor i (units):
   * Cap_A = 33,000,000
   * Cap_B = 160,000,000
   * Cap_C = 165,600,000
   * Cap_D = 12,000,000
   * Cap_E = 77,000,000 

**Decision Variables:**

* **x_ij:** Number of units (in thousands) purchased from vendor i in price range j
   * x_A: units from vendor A
   * x_B1: units from vendor B (22M-70M range)
   * x_B2: units from vendor B (70M+1-100M range)
   * x_B3: units from vendor B (100M+1-150M range)
   * x_B4: units from vendor B (150M+1-160M range)
   * x_C: units from vendor C
   * x_D: units from vendor D
   * x_E1: units from vendor E (0-42M range)
   * x_E2: units from vendor E (42M+1-77M range)

**Objective Function:**

Minimize total cost:

```
Minimize: 
S_A + S_B1*(x_B1 > 0) + S_B2*(x_B2 > 0) + S_B3*(x_B3 > 0) + S_B4*(x_B4 > 0) + S_C + S_D + S_E2*(x_E2 > 0)
 + C_A*x_A + C_B1*x_B1 + C_B2*x_B2 + C_B3*x_B3 + C_B4*x_B4 + C_C*x_C + C_D*x_D + C_E1*x_E1 
 + (C_E2_base - C_E2_discount*(x_E2 - 42000))*x_E2
```

**Constraints:**

1. **Demand Fulfillment:** 
   ```
   x_A + x_B1 + x_B2 + x_B3 + x_B4 + x_C + x_D + x_E1 + x_E2 = D/1000
   ``` 
2. **Capacity Constraints:**
   ```
   x_A <= Cap_A/1000
   x_B1 + x_B2 + x_B3 + x_B4 <= Cap_B/1000
   x_C <= Cap_C/1000
   x_D <= Cap_D/1000
   x_E1 + x_E2 <= Cap_E/1000
   ```
3. **Vendor B Minimum Order:**
   ```
   x_B1 >= 22000*(x_B1 > 0) 
   ```
4. **Price Range Constraints for Vendor B:**
   ```
   22000 <= x_B1 <= 70000
   70000 < x_B2 <= 100000
   100000 < x_B3 <= 150000
   150000 < x_B4 <= 160000 
   ```
5. **Price Range Constraints for Vendor E:**
   ```
   0 <= x_E1 <= 42000
   42000 < x_E2 <= 77000
   ```
6. **Non-negativity:**
   ```
   x_ij >= 0, for all i,j
   ```

This model captures the essential elements of the problem and can be solved using an optimization solver to determine the optimal order allocation that minimizes the total procurement cost. 


## 3. Generate the pyomo code

In [15]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [16]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Data
D = 239600480  # Total demand in units

S = {"A": 3855.34,
     "B1": 125804.84,
     "B2": 269304.84,
     "B3": 464304.84,
     "B4": 761304.84,
     "C": 13456.00,
     "D": 6583.98,
     "E2": 84000}  # Setup costs

C = {"A": 61.150,
     "B1": 68.099,
     "B2": 66.049,
     "B3": 64.099,
     "B4": 62.119,
     "C": 62.019,
     "D": 72.488,
     "E1": 70.150,
     "E2_base": 68.150}  # Unit costs

C_E2_discount = 0.0005  # Discount for E2

Cap = {"A": 33000000,
       "B": 160000000,
       "C": 165600000,
       "D": 12000000,
       "E": 77000000}  # Capacities

# Model
model = pyo.ConcreteModel()

# Sets
model.VENDORS = pyo.Set(initialize=["A", "B", "C", "D", "E"])
model.RANGES = pyo.Set(initialize=["B1", "B2", "B3", "B4", "E1", "E2"])  # Price ranges

# Variables
model.x = pyo.Var(model.VENDORS, model.RANGES, domain=pyo.NonNegativeReals)
# Sample data for x 
# In a real-world scenario, you would not pre-populate x
# This is solely for the purpose of showcasing the code execution. 
model.x["A", "B1"] = 10000
model.x["B", "B1"] = 22000
model.x["C", "B1"] = 10000
model.x["D", "B1"] = 10000
model.x["E", "E1"] = 40000


# Objective
def objective_rule(model):
    total_setup_cost = sum(S[i] * (model.x[i, j] > 0) for i in ["A", "B1", "B2", "B3", "B4", "C", "D", "E2"] for j in model.RANGES if (i, j) in model.x)
    total_variable_cost = C["A"] * model.x["A", "B1"] + \
                         C["B1"] * model.x["B", "B1"] + \
                         C["B2"] * model.x["B", "B2"] + \
                         C["B3"] * model.x["B", "B3"] + \
                         C["B4"] * model.x["B", "B4"] + \
                         C["C"] * model.x["C", "B1"] + \
                         C["D"] * model.x["D", "B1"] + \
                         C["E1"] * model.x["E", "E1"] + \
                         (C["E2_base"] - C_E2_discount * (model.x["E", "E2"] - 42000)) * model.x["E", "E2"]
    return total_setup_cost + total_variable_cost
model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
model.demand = pyo.Constraint(expr=sum(model.x[i, j] for i in model.VENDORS for j in model.RANGES if (i, j) in model.x) == D/1000)

model.capacity = pyo.ConstraintList()
model.capacity.add(model.x["A", "B1"] <= Cap["A"]/1000)
model.capacity.add(model.x["B", "B1"] + model.x["B", "B2"] + model.x["B", "B3"] + model.x["B", "B4"] <= Cap["B"]/1000)
model.capacity.add(model.x["C", "B1"] <= Cap["C"]/1000)
model.capacity.add(model.x["D", "B1"] <= Cap["D"]/1000)
model.capacity.add(model.x["E", "E1"] + model.x["E", "E2"] <= Cap["E"]/1000)

model.min_order_B = pyo.Constraint(expr=model.x["B", "B1"] >= 22000 * (model.x["B", "B1"] > 0))

model.range_B1 = pyo.Constraint(expr=22000 <= model.x["B", "B1"] <= 70000)
model.range_B2 = pyo.Constraint(expr=70000 < model.x["B", "B2"] <= 100000)
model.range_B3 = pyo.Constraint(expr=100000 < model.x["B", "B3"] <= 150000)
model.range_B4 = pyo.Constraint(expr=150000 < model.x["B", "B4"] <= 160000)

model.range_E1 = pyo.Constraint(expr=0 <= model.x["E", "E1"] <= 42000)
model.range_E2 = pyo.Constraint(expr=42000 < model.x["E", "E2"] <= 77000)

# Solve
solver = pyo.SolverFactory('cbc')  # Choose a solver, 'cbc' is a free open-source solver
results = solver.solve(model)

# Print results
print(results)
print("Optimal Solution:")
for i in model.VENDORS:
  for j in model.RANGES:
      if (i, j) in model.x:
        print(f"x[{i}, {j}] = {pyo.value(model.x[i, j])}")
print(f"Total Cost = {pyo.value(model.objective)}") 
```

## 4. Input problem data and try running the generated code

In [17]:
import pyomo.environ as pyo

# Data
D = 239600480  # Total demand in units

S = {"A": 3855.34,
     "B1": 125804.84,
     "B2": 269304.84,
     "B3": 464304.84,
     "B4": 761304.84,
     "C": 13456.00,
     "D": 6583.98,
     "E2": 84000}  # Setup costs

C = {"A": 61.150,
     "B1": 68.099,
     "B2": 66.049,
     "B3": 64.099,
     "B4": 62.119,
     "C": 62.019,
     "D": 72.488,
     "E1": 70.150,
     "E2_base": 68.150}  # Unit costs

C_E2_discount = 0.0005  # Discount for E2

Cap = {"A": 33000000,
       "B": 160000000,
       "C": 165600000,
       "D": 12000000,
       "E": 77000000}  # Capacities

# Model
model = pyo.ConcreteModel()

# Sets
model.VENDORS = pyo.Set(initialize=["A", "B", "C", "D", "E"])
model.RANGES = pyo.Set(initialize=["B1", "B2", "B3", "B4", "E1", "E2"])  # Price ranges

# Variables
model.x = pyo.Var(model.VENDORS, model.RANGES, domain=pyo.NonNegativeReals)
# Sample data for x 
# In a real-world scenario, you would not pre-populate x
# This is solely for the purpose of showcasing the code execution. 
model.x["A", "B1"] = 10000
model.x["B", "B1"] = 22000
model.x["C", "B1"] = 10000
model.x["D", "B1"] = 10000
model.x["E", "E1"] = 40000


# Objective
def objective_rule(model):
    total_setup_cost = sum(S[i] * (model.x[i, j] > 0) for i in ["A", "B1", "B2", "B3", "B4", "C", "D", "E2"] for j in model.RANGES if (i, j) in model.x)
    total_variable_cost = C["A"] * model.x["A", "B1"] + \
                         C["B1"] * model.x["B", "B1"] + \
                         C["B2"] * model.x["B", "B2"] + \
                         C["B3"] * model.x["B", "B3"] + \
                         C["B4"] * model.x["B", "B4"] + \
                         C["C"] * model.x["C", "B1"] + \
                         C["D"] * model.x["D", "B1"] + \
                         C["E1"] * model.x["E", "E1"] + \
                         (C["E2_base"] - C_E2_discount * (model.x["E", "E2"] - 42000)) * model.x["E", "E2"]
    return total_setup_cost + total_variable_cost
model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
model.demand = pyo.Constraint(expr=sum(model.x[i, j] for i in model.VENDORS for j in model.RANGES if (i, j) in model.x) == D/1000)

model.capacity = pyo.ConstraintList()
model.capacity.add(model.x["A", "B1"] <= Cap["A"]/1000)
model.capacity.add(model.x["B", "B1"] + model.x["B", "B2"] + model.x["B", "B3"] + model.x["B", "B4"] <= Cap["B"]/1000)
model.capacity.add(model.x["C", "B1"] <= Cap["C"]/1000)
model.capacity.add(model.x["D", "B1"] <= Cap["D"]/1000)
model.capacity.add(model.x["E", "E1"] + model.x["E", "E2"] <= Cap["E"]/1000)

model.min_order_B = pyo.Constraint(expr=model.x["B", "B1"] >= 22000 * (model.x["B", "B1"] > 0))

model.range_B1 = pyo.Constraint(expr=22000 <= model.x["B", "B1"] <= 70000)
model.range_B2 = pyo.Constraint(expr=70000 < model.x["B", "B2"] <= 100000)
model.range_B3 = pyo.Constraint(expr=100000 < model.x["B", "B3"] <= 150000)
model.range_B4 = pyo.Constraint(expr=150000 < model.x["B", "B4"] <= 160000)

model.range_E1 = pyo.Constraint(expr=0 <= model.x["E", "E1"] <= 42000)
model.range_E2 = pyo.Constraint(expr=42000 < model.x["E", "E2"] <= 77000)

# Solve
solver = pyo.SolverFactory('cbc')  # Choose a solver, 'cbc' is a free open-source solver
results = solver.solve(model)

# Print results
print(results)
print("Optimal Solution:")
for i in model.VENDORS:
  for j in model.RANGES:
      if (i, j) in model.x:
        print(f"x[{i}, {j}] = {pyo.value(model.x[i, j])}")
print(f"Total Cost = {pyo.value(model.objective)}") 

ERROR: Rule failed when generating expression for Objective objective with
index None: TypeError: unsupported operand type(s) for *: 'float' and
'InequalityExpression'
ERROR: Constructing component 'objective' from data=None failed:
        TypeError: unsupported operand type(s) for *: 'float' and
        'InequalityExpression'


TypeError: unsupported operand type(s) for *: 'float' and 'InequalityExpression'

## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [18]:
print(response.text)

## Mathematical Optimization Model for Supplier Selection

**Parameters:**

* **D:** Total demand (units): 239,600,480 
* **S_i:** Setup cost for vendor i (i = A, B, C, D, E)
   * S_A = $3855.34
   * S_B1 = $125,804.84 (22M - 70M units)
   * S_B2 = $269,304.84 (70M+1 - 100M units)
   * S_B3 = $464,304.84 (100M+1 - 150M units)
   * S_B4 = $761,304.84 (150M+1 - 160M units)
   * S_C = $13,456.00
   * S_D = $6,583.98
   * S_E1 = $0 (0 - 42M units)
   * S_E2 = $84,000 (42M+1 - 77M units)
* **C_ij:** Unit cost per thousand units for vendor i in price range j 
   * C_A = $61.150
   * C_B1 = $68.099 (22M - 70M units)
   * C_B2 = $66.049 (70M+1 - 100M units)
   * C_B3 = $64.099 (100M+1 - 150M units)
   * C_B4 = $62.119 (150M+1 - 160M units)
   * C_C = $62.019
   * C_D = $72.488
   * C_E1 = $70.150 (0 - 42M units)
   * C_E2_base = $68.150 (42M+1 - 77M units)
   * C_E2_discount = 0.0005 (per thousand units over 42M)
* **Cap_i:** Maximum supply capacity for vendor i (units):
   * Cap_A = 33,000,00

In [20]:
print(response2.text)

```python
import pyomo.environ as pyo

# Data
D = 239600480  # Total demand in units

S = {"A": 3855.34,
     "B1": 125804.84,
     "B2": 269304.84,
     "B3": 464304.84,
     "B4": 761304.84,
     "C": 13456.00,
     "D": 6583.98,
     "E2": 84000}  # Setup costs

C = {"A": 61.150,
     "B1": 68.099,
     "B2": 66.049,
     "B3": 64.099,
     "B4": 62.119,
     "C": 62.019,
     "D": 72.488,
     "E1": 70.150,
     "E2_base": 68.150}  # Unit costs

C_E2_discount = 0.0005  # Discount for E2

Cap = {"A": 33000000,
       "B": 160000000,
       "C": 165600000,
       "D": 12000000,
       "E": 77000000}  # Capacities

# Model
model = pyo.ConcreteModel()

# Sets
model.VENDORS = pyo.Set(initialize=["A", "B", "C", "D", "E"])
model.RANGES = pyo.Set(initialize=["B1", "B2", "B3", "B4", "E1", "E2"])  # Price ranges

# Variables
model.x = pyo.Var(model.VENDORS, model.RANGES, domain=pyo.NonNegativeReals)
# Sample data for x 
# In a real-world scenario, you would not pre-populate x
# This is solel